In [ ]:
import os.path as osp
import sys
import random

path = osp.dirname(osp.abspath(''))
sys.path.append(path)
sys.path.append(osp.join(path, "./open_biomed"))
path

In [ ]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

In [ ]:

def read_text(path=None):
    with open(path, 'r') as f:

        text_list = []
        for index, line in enumerate(f):
            text = line.strip()
            text_list.append(text)
    return text_list
path='./text_after_openai_chemistry_extracted_cleaned.txt'
extracted_text=read_text(path=path)
path='./text_after_structure_filter.txt'
texts=read_text(path=path)

# path='./smis_after_structure_filter.txt'
# smis=read_text(path=path)

In [ ]:
def structure_isomers(path=''):
    sim_list=[]
    with open(path, 'r') as f:
        for index, line in enumerate(f):

            isomers_list = line.strip().split("\t")
            sim_list.append(isomers_list[0])
    return sim_list
structure_isomers=structure_isomers(path='./tautomers_ismers.txt')

smis=structure_isomers
print((len(texts),len(smis)))

In [ ]:
len(extracted_text),len(texts),len(smis)

In [ ]:
# 线性核函数，用 PyTorch 实现
def linear_kernel(X):
    return torch.mm(X, X.T)  # 使用矩阵乘法

# 居中核矩阵
def center_kernel(K):
    n = K.shape[0]
    H = torch.eye(n, device=K.device) - torch.ones((n, n), device=K.device) / n  # 中心化矩阵 H
    return torch.mm(H, torch.mm(K, H))

# CKA 实现，计算两个张量 X 和 Y 的 CKA
def CKA(X, Y):
    # 计算线性核矩阵
    K = linear_kernel(X)
    L = linear_kernel(Y)

    # 居中核矩阵
    Kc = center_kernel(K)
    Lc = center_kernel(L)

    # 计算 HSIC 值
    hsic_XY = torch.trace(torch.mm(Kc, Lc))
    hsic_XX = torch.trace(torch.mm(Kc, Kc))
    hsic_YY = torch.trace(torch.mm(Lc, Lc))

    # 计算 CKA 值
    cka_value = hsic_XY / torch.sqrt(hsic_XX * hsic_YY)




    return round(cka_value.item(),3)

    

In [ ]:
from metrics import cka,gram_linear
# structure_feats_np = structure_feats[indes].cpu().detach().numpy()
# text_feats_np = text_feats[indes].cpu().detach().numpy()
# # Calculate Gram matrices for structure_feats and text_feats
def de_biased_cka(structure_feats,text_feats):
    structure_feats_np = structure_feats.cpu().detach().numpy()
    text_feats_np = text_feats.cpu().detach().numpy()
    gram_structure = gram_linear(structure_feats_np)
    gram_text = gram_linear(text_feats_np)

    # Calculate CKA between the two Gram matrices
    cka_value = cka(gram_structure, gram_text, debiased=True)

    return round(cka_value,3)

In [ ]:
import torch.nn.functional as F

def retrieve(A, B, K=1):
    """
    检索任务函数，输入两个 tensor A 和 B，返回 A 和 B 匹配的索引值、匹配的分数和正确匹配的条目数。

    参数：
    - A: torch.Tensor，待匹配的 query tensor，形状 (n, d)。
    - B: torch.Tensor，待匹配的 target tensor，形状 (n, d)。
    - K: int，检索 top K 匹配。

    返回：
    - indices: 每个 A 对应的最相似 B 的索引值，形状 (n, K)。
    - scores: 每个 A 对应的最相似 B 的分数，形状 (n, K)。
    - correct_total: 匹配正确的总条目数。
    """

    # 计算 A 和 B 之间的余弦相似度 (n, n)
    similarities = F.cosine_similarity(A.unsqueeze(1), B.unsqueeze(0), dim=-1)
    # 获取相似度最高的前 K 个条目的索引和分数
    topk_scores, topk_indices = torch.topk(similarities, K, dim=1)
    # 计算总的正确匹配数量
    # 正确的定义：A[i] 和 B[i] 相匹配，意味着第 i 个 A 应该匹配第 i 个 B
    correct_total = 0
    n = A.size(0)
    for i in range(n):
        # 检查 top K 的索引是否包含正确匹配
        if i in topk_indices[i]:
            correct_total += 1
    return topk_indices, topk_scores, correct_total

In [ ]:
def overleap(indexA,indexB ):
    C=[]
    D=[]
    for a, b in zip(indexA, indexB):
        overlap = list(set(a) & set(b))
        C.append(overlap)
        D.append([len(overlap)])

    # Print results
    print("Overlap C:", C)
    print("Number of overlaps D:", D)

    sum=0
    for i in C:
        sum=sum+len(i)
    print(sum)
    jaccard_similarity=0
    for a, b in zip(indexA, indexB):
        jaccard_similarity = jaccard_similarity+len(set(a) & set(b)) / len(set(a) | set(b))

    print("Mean Jaccard Similarity:", round(jaccard_similarity/len(indexA),3))

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

def KM(embedding=None,num_clusters=5):
    num_clusters = num_clusters  # You can set this based on your requirement

    # Initialize and fit the KMeans model
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(embedding)

    return kmeans

In [ ]:
from sklearn.neighbors import NearestNeighbors

def NN(embedding=None,num=10):
    # Initialize the Nearest Neighbors model
    num_neighbors = num  # Number of nearest neighbors to find
    nn_model = NearestNeighbors(n_neighbors=num_neighbors)

    # Fit the model to your data
    try:
        nn_model.fit(embedding.cpu().numpy())
    except:
        nn_model.fit(embedding.detach().cpu().numpy())

    return nn_model

# from sklearn.neighbors import NearestNeighbors

# def NN(embedding=None, num=10):
#     """
#     Initialize and fit a Nearest Neighbors model using cosine similarity.

#     Parameters:
#         embedding (torch.Tensor or np.ndarray): The data to fit the model on.
#         num (int): Number of nearest neighbors to find.

#     Returns:
#         NearestNeighbors: A fitted Nearest Neighbors model.
#     """
#     # Number of nearest neighbors to find
#     num_neighbors = num  

#     # Initialize the Nearest Neighbors model with cosine similarity
#     nn_model = NearestNeighbors(n_neighbors=num_neighbors, metric='cosine')

#     # Fit the model to your data
#     try:
#         nn_model.fit(embedding.cpu().numpy())
#     except AttributeError:  # Handle cases where `cpu` is not available
#         nn_model.fit(embedding.detach().cpu().numpy())

#     return nn_model


In [ ]:
len(extracted_text),len(smis),len(texts)

## MolFM

In [ ]:
# prepare data
import json
from open_biomed.utils.data_utils import DataProcessorFast
from open_biomed.utils import fix_path_in_config

config = json.load(open("./../configs/mtr/molfm.json", "r"))
# config['network']['bert_config_path']='../../configs/encoders/multimodal/molfm_bert_config.json'
# config['data']['mol']['featurizer']['text']['model_name_or_path']='../../ckpts/text_ckpts/scibert_scivocab_uncased'
fix_path_in_config(config, path)
print("Config: ", config)
processor = DataProcessorFast(entity_type="molecule", config=config["data"]["mol"])
processor.featurizer.set_mol2text_dict(dict(zip(smis, texts)))
mols = processor(smis)

In [ ]:
import torch
from open_biomed.models.multimodal import MolFM
model = MolFM(config["network"])
# state_dict = torch.load("./../ckpts/fusion_ckpts/MolFM.pth", map_location="cpu")["model"]
# model.load_state_dict(state_dict, strict=False)
model.eval()


In [ ]:
model.to(device)
with torch.no_grad():
    structure_feats = model.encode_mol(mols["structure"].to(device), proj=True)
    text_feats = model.encode_text(mols["text"].to(device))

### with original data 

In [ ]:
structure_feats.shape,text_feats.shape

### CKA

In [ ]:
CKA(structure_feats,text_feats)

In [ ]:
de_biased_cka(structure_feats,text_feats)

### match

In [ ]:
A =structure_feats
B = text_feats
indices_k1, scores_k1, correct_total_k1 = retrieve(A.to(device), B.to(device), K=1)
# 打印结果
# print("Top K 索引：", indices_k1)
# print("Top K 分数：", scores_k1)
print("正确匹配的条目数：", correct_total_k1)

In [ ]:
A =structure_feats
B = text_feats
indices, scores, correct_total = retrieve(A.to(device), B.to(device), K=10)
# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", correct_total)

### text NN

In [ ]:
texts_NN=NN(embedding=B)
texts_NN_distance,texts_NN_indes=texts_NN.kneighbors(B.cpu().detach())
overleap(indices.cpu().numpy(), texts_NN_indes)

### SMILES NN

In [ ]:
smis_NN=NN(embedding=A)
smis_NN_distance,smis_NN_indes=smis_NN.kneighbors(A.cpu().detach())
overleap(indices.cpu().numpy(), smis_NN_indes)

### K-means
### texts

In [ ]:
# K-means
texts_KM=KM(B.cpu().numpy())
cls_index=texts_KM.predict(B.cpu().numpy())

C=B[indices_k1.cpu()].reshape(B.shape).cpu().numpy()
match_index=texts_KM.predict(C)
x=cls_index==match_index
np.sum(x)

In [ ]:
2896/3515

##### smis

In [ ]:
# A =structure_feats
# B = text_feats

smis_KM=KM(A.cpu().numpy())

smis_index=smis_KM.predict(A.cpu().numpy())

C=A[indices_k1.cpu()].reshape(A.shape).cpu().numpy()
match_index=smis_KM.predict(C)

x=smis_index==match_index

np.sum(x)

### with extracted data

In [ ]:
# prepare data
import json
from open_biomed.utils.data_utils import DataProcessorFast
from open_biomed.utils import fix_path_in_config

config = json.load(open("./../configs/mtr/molfm.json", "r"))
# config['network']['bert_config_path']='../../configs/encoders/multimodal/molfm_bert_config.json'
# config['data']['mol']['featurizer']['text']['model_name_or_path']='../../ckpts/text_ckpts/scibert_scivocab_uncased'

fix_path_in_config(config, path)
print("Config: ", config)
processor = DataProcessorFast(entity_type="molecule", config=config["data"]["mol"])
processor.featurizer.set_mol2text_dict(dict(zip(smis, extracted_text)))
mols = processor(smis)

In [ ]:
import torch
from open_biomed.models.multimodal import MolFM
model = MolFM(config["network"])
# state_dict = torch.load("./../ckpts/fusion_ckpts/MolFM.pth", map_location="cpu")["model"]
# model.load_state_dict(state_dict, strict=False)
model.eval()


In [ ]:
model.to(device)
with torch.no_grad():
    structure_feats = model.encode_mol(mols["structure"].to(device), proj=True)
    text_feats = model.encode_text(mols["text"].to(device))



### CKA

In [ ]:
de_biased_cka(structure_feats,text_feats)

In [ ]:
CKA(structure_feats,text_feats)

### MATCH

In [ ]:
A =structure_feats
B = text_feats
indices_k1, scores_k1, correct_total_k1 = retrieve(A.to(device), B.to(device), K=1)
# 打印结果
# print("Top K 索引：", indices_k1)
# print("Top K 分数：", scores_k1)
print("正确匹配的条目数：", correct_total_k1)

In [ ]:
A =structure_feats
B = text_feats
indices, scores, correct_total = retrieve(A.to(device), B.to(device), K=10)
# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", correct_total)

### text NN

In [ ]:
texts_NN=NN(embedding=B)
texts_NN_distance,texts_NN_indes=texts_NN.kneighbors(B.cpu().detach())
overleap(indices.cpu().numpy(), texts_NN_indes)

### SMILES NN

In [ ]:
smis_NN=NN(embedding=A)
smis_NN_distance,smis_NN_indes=smis_NN.kneighbors(A.cpu().detach())
overleap(indices.cpu().numpy(), smis_NN_indes)

### K-means
### texts

In [ ]:
# K-means
texts_KM=KM(B.cpu().numpy())
cls_index=texts_KM.predict(B.cpu().numpy())

C=B[indices_k1.cpu()].reshape(B.shape).cpu().numpy()
match_index=texts_KM.predict(C)
x=cls_index==match_index
np.sum(x)

##### smis

In [ ]:
# A =structure_feats
# B = text_feats

smis_KM=KM(A.cpu().numpy())

smis_index=smis_KM.predict(A.cpu().numpy())

C=A[indices_k1.cpu()].reshape(A.shape).cpu().numpy()
match_index=smis_KM.predict(C)

x=smis_index==match_index

np.sum(x)

## MoMu

In [ ]:
# prepare data
import json
from open_biomed.utils.data_utils import DataProcessorFast
from open_biomed.utils import fix_path_in_config

config = json.load(open("./../configs/mtr/momu.json", "r"))
# config['data']['mol']['featurizer']['text']['model_name_or_path']='../../ckpts/text_ckpts/scibert_scivocab_uncased'

fix_path_in_config(config, path)
print("Config: ", config)
# print(config["data"]["mol"])
processor = DataProcessorFast(entity_type="molecule", config=config["data"]["mol"])
processor.featurizer.set_mol2text_dict(dict(zip(smis, texts)))
mols = processor(smis)


# config = json.load(open("./../configs/mtr/molfm.json", "r"))
# # config['network']['bert_config_path']='../../configs/encoders/multimodal/molfm_bert_config.json'
# # config['data']['mol']['featurizer']['text']['model_name_or_path']='../../ckpts/text_ckpts/scibert_scivocab_uncased'
# fix_path_in_config(config, path)
# print("Config: ", config)
# processor = DataProcessorFast(entity_type="molecule", config=config["data"]["mol"])
# processor.featurizer.set_mol2text_dict(dict(zip(smis, texts)))
# mols = processor(smis)

In [ ]:

from open_biomed.models.multimodal import MoMu
model = MoMu(config["network"])
# state_dict = torch.load("../ckpts/fusion_ckpts/MoMu-S.ckpt", map_location="cuda")
# print(state_dict)
# state_dict = torch.load("../ckpts/fusion_ckpts/MoMu-K.ckpt", map_location="cuda")["model"]
# model.load_state_dict(state_dict)
# model.eval()

# with torch.no_grad():
#     structure_feats = model.encode_mol(mols["structure"], proj=True)
#     text_feats = model.encode_text(mols["text"])

# for i in range(len(smis)):
#     similarity = torch.cosine_similarity(structure_feats[i], text_feats)
#     best = torch.argmax(similarity).item()
#     print("Similarity for ", smis[i], "is", similarity.numpy(), ", Retrieved text is \"", texts[best], "\"")

In [ ]:
model.load_state_dict(state_dict['state_dict'], strict=False)

In [ ]:
# model.eval()

# with torch.no_grad():
#     structure_feats = model.encode_mol(mols["structure"], proj=True)
#     text_feats = model.encode_text(mols["text"])

In [ ]:
model.to(device)
with torch.no_grad():
    structure_feats = model.encode_mol(mols["structure"].to(device), proj=True)
    text_feats = model.encode_text(mols["text"].to(device))

### with original data 

In [ ]:
structure_feats.shape,text_feats.shape

### CKA

In [ ]:
CKA(structure_feats,text_feats)

In [ ]:
de_biased_cka(structure_feats,text_feats)

### match

In [ ]:
A =structure_feats
B = text_feats
indices_k1, scores_k1, correct_total_k1 = retrieve(A.to(device), B.to(device), K=1)
# 打印结果
# print("Top K 索引：", indices_k1)
# print("Top K 分数：", scores_k1)
print("正确匹配的条目数：", correct_total_k1)

In [ ]:
A =structure_feats
B = text_feats
indices, scores, correct_total = retrieve(A.to(device), B.to(device), K=10)
# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", correct_total)

### text NN

In [ ]:
texts_NN=NN(embedding=B)
texts_NN_distance,texts_NN_indes=texts_NN.kneighbors(B.cpu().detach())
overleap(indices.cpu().numpy(), texts_NN_indes)

### SMILES NN

In [ ]:
smis_NN=NN(embedding=A)
smis_NN_distance,smis_NN_indes=smis_NN.kneighbors(A.cpu().detach())
overleap(indices.cpu().numpy(), smis_NN_indes)

### K-means
### texts

In [ ]:
# K-means
texts_KM=KM(B.cpu().numpy())
cls_index=texts_KM.predict(B.cpu().numpy())

C=B[indices_k1.cpu()].reshape(B.shape).cpu().numpy()
match_index=texts_KM.predict(C)
x=cls_index==match_index
np.sum(x)

In [ ]:
2718/3515

##### smis

In [ ]:
# A =structure_feats
# B = text_feats

smis_KM=KM(A.cpu().numpy())

smis_index=smis_KM.predict(A.cpu().numpy())

C=A[indices_k1.cpu()].reshape(A.shape).cpu().numpy()
match_index=smis_KM.predict(C)

x=smis_index==match_index

np.sum(x)

### with extracted text

In [ ]:
# prepare data
import json
from open_biomed.utils.data_utils import DataProcessorFast
from open_biomed.utils import fix_path_in_config

config = json.load(open("./../configs/mtr/momu.json", "r"))
# config['data']['mol']['featurizer']['text']['model_name_or_path']='../../ckpts/text_ckpts/scibert_scivocab_uncased'

fix_path_in_config(config, path)
print("Config: ", config)
processor = DataProcessorFast(entity_type="molecule", config=config["data"]["mol"])
processor.featurizer.set_mol2text_dict(dict(zip(smis, extracted_text)))
mols = processor(smis)

In [ ]:
model.to(device)
with torch.no_grad():
    structure_feats = model.encode_mol(mols["structure"].to(device), proj=True)
    text_feats = model.encode_text(mols["text"].to(device))

In [ ]:
structure_feats.shape,text_feats.shape

### CKA

In [ ]:
CKA(structure_feats,text_feats)

In [ ]:
de_biased_cka(structure_feats,text_feats)

### match

In [ ]:
A =structure_feats
B = text_feats
indices_k1, scores_k1, correct_total_k1 = retrieve(A.to(device), B.to(device), K=1)
# 打印结果
# print("Top K 索引：", indices_k1)
# print("Top K 分数：", scores_k1)
print("正确匹配的条目数：", correct_total_k1)

In [ ]:
A =structure_feats
B = text_feats
indices, scores, correct_total = retrieve(A.to(device), B.to(device), K=10)
# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", correct_total)

### text NN

In [ ]:
texts_NN=NN(embedding=B)
texts_NN_distance,texts_NN_indes=texts_NN.kneighbors(B.cpu().detach())
overleap(indices.cpu().numpy(), texts_NN_indes)

### SMILES NN

In [ ]:
smis_NN=NN(embedding=A)
smis_NN_distance,smis_NN_indes=smis_NN.kneighbors(A.cpu().detach())
overleap(indices.cpu().numpy(), smis_NN_indes)

### K-means
### texts

In [ ]:
# K-means
texts_KM=KM(B.cpu().numpy())
cls_index=texts_KM.predict(B.cpu().numpy())

C=B[indices_k1.cpu()].reshape(B.shape).cpu().numpy()
match_index=texts_KM.predict(C)
x=cls_index==match_index
np.sum(x)

In [ ]:
2732/3515

##### smis

In [ ]:
# A =structure_feats
# B = text_feats

smis_KM=KM(A.cpu().numpy())

smis_index=smis_KM.predict(A.cpu().numpy())

C=A[indices_k1.cpu()].reshape(A.shape).cpu().numpy()
match_index=smis_KM.predict(C)

x=smis_index==match_index

np.sum(x)

## CLAMP

In [ ]:
import torch
import clamp

model = clamp.CLAMP(device='cpu')
model.eval()



In [ ]:
# len(extracted_text),len(smis),len(texts)

text_feats=model.encode_text(texts).to(device)

structure_feats=model.encode_smiles(smis).to(device)

In [ ]:
len(structure_feats),len(text_feats)

In [ ]:
structure_feats.shape,text_feats.shape

### CKA

In [ ]:
CKA(structure_feats,text_feats)

In [ ]:
de_biased_cka(structure_feats,text_feats)

### match

In [ ]:
A =structure_feats
B = text_feats
indices_k1, scores_k1, correct_total_k1 = retrieve(A.to('cpu'), B.to('cpu'), K=1)
# 打印结果
# print("Top K 索引：", indices_k1)
# print("Top K 分数：", scores_k1)
print("正确匹配的条目数：", correct_total_k1)

In [ ]:
A =structure_feats
B = text_feats
indices, scores, correct_total = retrieve(A.to('cpu'), B.to('cpu'), K=10)
# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", correct_total)

### text NN

In [ ]:
texts_NN=NN(embedding=B)
texts_NN_distance,texts_NN_indes=texts_NN.kneighbors(B.cpu().detach())
overleap(indices.cpu().numpy(), texts_NN_indes)

### SMILES NN

In [ ]:
smis_NN=NN(embedding=A)
smis_NN_distance,smis_NN_indes=smis_NN.kneighbors(A.cpu().detach())
overleap(indices.cpu().numpy(), smis_NN_indes)

### K-means
### texts

In [ ]:
# K-means
texts_KM=KM(B.cpu().numpy())
cls_index=texts_KM.predict(B.cpu().numpy())

C=B[indices_k1.cpu()].reshape(B.shape).cpu().numpy()
match_index=texts_KM.predict(C)
x=cls_index==match_index
np.sum(x)

In [ ]:
2718/3515

##### smis

In [ ]:
# A =structure_feats
# B = text_feats

smis_KM=KM(A.cpu().numpy())

smis_index=smis_KM.predict(A.cpu().numpy())

C=A[indices_k1.cpu()].reshape(A.shape).cpu().numpy()
match_index=smis_KM.predict(C)

x=smis_index==match_index

np.sum(x)

### with extracted text

In [ ]:
text_feats=model.encode_text(extracted_text).to(device)


In [ ]:
structure_feats.shape,text_feats.shape

### CKA

In [ ]:
CKA(structure_feats,text_feats)

In [ ]:
de_biased_cka(structure_feats,text_feats)

### match

In [ ]:
A =structure_feats
B = text_feats
indices_k1, scores_k1, correct_total_k1 = retrieve(A.to('cpu'), B.to('cpu'), K=1)
# 打印结果
# print("Top K 索引：", indices_k1)
# print("Top K 分数：", scores_k1)
print("正确匹配的条目数：", correct_total_k1)

In [ ]:
A =structure_feats
B = text_feats
indices, scores, correct_total = retrieve(A.to('cpu'), B.to('cpu'), K=10)
# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", correct_total)

### text NN

In [ ]:
texts_NN=NN(embedding=B)
texts_NN_distance,texts_NN_indes=texts_NN.kneighbors(B.cpu().detach())
overleap(indices.cpu().numpy(), texts_NN_indes)

### SMILES NN

In [ ]:
smis_NN=NN(embedding=A)
smis_NN_distance,smis_NN_indes=smis_NN.kneighbors(A.cpu().detach())
overleap(indices.cpu().numpy(), smis_NN_indes)

### K-means
### texts

In [ ]:
# K-means
texts_KM=KM(B.cpu().numpy())
cls_index=texts_KM.predict(B.cpu().numpy())

C=B[indices_k1.cpu()].reshape(B.shape).cpu().numpy()
match_index=texts_KM.predict(C)
x=cls_index==match_index
np.sum(x)

In [ ]:
2718/3515

##### smis

In [ ]:
# A =structure_feats
# B = text_feats

smis_KM=KM(A.cpu().numpy())

smis_index=smis_KM.predict(A.cpu().numpy())

C=A[indices_k1.cpu()].reshape(A.shape).cpu().numpy()
match_index=smis_KM.predict(C)

x=smis_index==match_index

np.sum(x)